In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
source = pd.read_csv('/content/drive/MyDrive/파일위치/gujo_qna_kkm_sbert.csv')
df = pd.read_csv('/content/drive/MyDrive/파일위치/pan_qna_kkm_sbert.csv')
source = pd.concat([source, df], axis=0).reset_index(drop=True)
source

,question,answer,embedding
0,최저 임금 제도 란 무엇 며 최저 임 금액 어떻 게 결정 는지요,최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는...,[-5.09301648e-02 -3.32413241e-02 1.03755467e-...
1,상시 근로 자수 인인 회사 년 월 부터 근무 다가 개월 ...,퇴직금제도에 관하여 「근로기준법」제34조에 의하면 「근로자퇴직급여 보장법」이 정하는...,[-3.77310514e-01 -4.29380871e-02 5.94415545e-...
2,회사 입사 년 째 더 해 소속 부서 업무 회사 독립 자 회사 일방...,기업의 합병·분할·영업양도 등의 경우 근로자들이 조직변경 전후에 계속하여 근무를 하...,[-3.89823109e-01 1.80484757e-01 8.53554010e-...
3,학습지 제작 판매 회사 회사 위탁 업무 계약 체결 교육 상담 교사 ...,「근로기준법」상 근로자의 범위에 관하여 판례는 “근로기준법상의 근로자에 해당하는지 ...,[-2.00027689e-01 3.72830212e-01 5.04132569e-...
4,대학 병원 전공의 과정 밟 레지던트 니다 그러 데 몇 달 전 부터 ...,「근로기준법」제2조 제1항 1호에서는 ‘근로자’를 ‘직업의 종류와 관계없이 임금을 ...,[-2.77250111e-02 1.06599852e-01 5.86231887e-...
...,...,...,...
102853,모집 설립 어서 설립 경과 조사 절차 어서 설명 주 시 시오,모집설립에 있어서 설립경과의 조사는 발기설립의 경우와 거의 동일합니다. 모집설립에 ...,[-2.38549814e-01 2.11867094e-02 9.72366035e-...
102854,발기 설립 어서 변태 설립 사항 조사 설명 주 시 시오,발기설립에 있어서 정관으로 상법 제290조 각 호의 변태설립사항을 정한 때에는 이사...,[-2.32459411e-01 -4.19367775e-02 1.01206744e+...
102855,발기 설립 어서 설립 경과 조사 절차 설명 주 시 시오,이사와 감사는 취임후 지체없이 회사의 설립에 관한 모든 사항이 법령 또는 정관의 규...,[-4.16801900e-01 -7.41601065e-02 1.07209158e+...
102856,발기 설립 임원 선임 절차 어떻 게 는가요,발기인이 인수한 주식에 대하여 상법 제295조의 규정에 의한 납입과 현물출자의 이행...,[ 7.15333372e-02 6.66695386e-02 9.00789678e-...


In [ ]:
source.to_csv('/content/drive/MyDrive/파일위치/pan_gujo_qna_kkm_sbert.csv', index=False)

In [ ]:
def str_to_vector(emd):
    res = emd.split(' ')
    #print(emd)
    #print(len(r))
    r = []
    for a in res:
        if len(a) >2:
            r.append(a)
    if len(r) != 768:
        print(len(r))
    for i in range(len(r)):
        r[i] = r[i].replace('[', '')
        r[i] = r[i].replace(']', '')
        r[i] = r[i].replace('\n', '')
        r[i] = float(r[i])
    return np.array(r)

In [ ]:
import numpy as np
emd2 = source.apply(lambda x: str_to_vector(x['embedding']), axis=1)
answer = source['answer']
question = source['question']

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Kkma

kkma = Kkma()

In [ ]:
from tqdm import tqdm
import re
stopwords = ['하', '의', '에', '이', '는', '을', '가', '되', '를', '있', '여', '으로', '경우', '는가', '수', '아', '로', '그', '은', '고', '제', '것', '지', '에서', '저', '대하', '조', '어', '등', '행위', '관하', '해당', '보', '항', '에게', '및', '과', '받', '와', '없', '의하', '다고', '처분', '않', '또는', '였', '었', '나요', '따르']


def tokenize_sentence(sentence):
    tokenized_sentence = []
    tokenized_sentence = kkma.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    l = ''
    for s in stopwords_removed_sentence:
        s = re.sub(r"[^가-힣\s]", " ", s)
        l += s +' '
    return l

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
from math import *

def jaccard_similarity(x, y):
    intersection_cardinality = set(x).intersection(set(y))
    union_cardinality = set(x).union(set(y))

    return len(intersection_cardinality) / len(union_cardinality)

In [ ]:
def return_answer(q):
    q = tokenize_sentence(q)
    embedding = model.encode(q)
    #유사도 계산 및 정렬
    sim_scores = [[k, 0.1*jaccard_similarity(question[k], q)+0.9*cos_sim(emd2[k], embedding)] for k in range(len(source))]
    sim_scores.sort(key=lambda x: x[1], reverse=True) #sim_scores의 각 리스트 중 두번째 요소를 정렬 기준으로

    #상위 다섯 개 저장(중복 제거)
    result = [] #전체
    score = []
    for sim in sim_scores:
        a = answer[sim[0]]
        if a not in result:
            result.append(a)
            score.append(sim[1])
        else:
            #print(a)
            pass
        if len(result) == 5:
            break
    print(score)
    return result

In [ ]:
return_answer('사실로 인한 명예훼손죄가 성립되는지?')

[0.8385475897718766, 0.8183368555488779, 0.8173528564960504, 0.8173255498620453, 0.8135115583811161]


['명예훼손의 경우에 관하여 「민법」 제764조는 “타인의 명예를 훼손한 자에 대하여는 법원은 피해자의 청구에 의하여 손해배상에 갈음하거나 손해배상과 함께 명예회복에 적당한 처분을 명할 수 있다.”라고 규정하고 있습니다.그런데 이러한 규정이 종중 등 법인 아닌 사단에도 적용되어 종중이 명예를 훼손당한 경우 위 규정에 의하여 손해배상을 청구하거나 명예회복에 적당한 처분을 청구할 수 있는지 문제됩니다.이에 관하여 판례는 “민법 제764조에서 말하는 명예라 함은 사람의 품성, 덕행, 명예, 신용 등 세상으로부터 받는 객관적인 평가를 말하는 것이고, 특히 법인의 경우에는 그 사회적 명예, 신용을 가리키는데 다름없는 것으로 명예를 훼손한다는 것은 그 사회적 평가를 침해하는 것을 말하고, 이와 같은 법인의 명예가 훼손된 경우에 그 법인은 상대방에 대하여 불법행위로 인한 손해배상과 함께 명예회복에 적당한 처분을 청구할 수 있고, 종중과 같이 소송상 당사자능력이 있는 비법인사단 역시 마찬가지이다.”라고 하였으며, “사람(종중 등의 경우에도 마찬가지이다.)이 갖는 명예에 관한 권리는 일종의 인격권으로 볼 수 있는 것으로서, 그 성질상 일단 침해된 후에는 금전배상이나 명예회복에 필요한 처분 등의 구제수단만으로는 그 피해의 완전한 회복이 어렵고 손해전보의 실효성을 기대하기 어려우므로, 이와 같은 인격권의 침해에 대하여는 사전예방적 구제수단으로 침해행위의 정지·방지 등의 금지청구권이 인정될 수 있다.”라고 하였습니다(대법원 1997. 10. 24. 선고 96다17851 판결).또한, “법인의 목적사업 수행에 영향을 미칠 정도로 법인의 사회적 명성, 신용을 훼손하여 법인의 사회적 평가가 침해된 경우에는 그 법인에 대하여 불법행위를 구성한다.”라고 하였습니다(대법원 1999. 10. 22. 선고 98다6381 판결).따라서 종중 등 비법인 사단도 명예가 훼손될 경우에는 그 손해배상 및 명예회복에 적당한 처분을 청구할 수 있다고 할 것입니다.\n',
 '범죄사실을 부인하거나 죄의 뉘우침

In [ ]:
return_answer('건물 소유자의 배우자와 한 임대차 계약의 효력')

[0.7713995668788729, 0.7616654262000736, 0.7613287066956059, 0.7574867239594474, 0.7462614194488738]


['주택임대차보호법이 적용되는 임대차가 임차인과 주택의 소유자인 임대인 사이에 임대차계약이 체결된 경우로 한정되는 것은 아닙니다.',
 '주임법 제2조는 \'이 법은 주거용 건물(이하 "주택"이라 한다)의 전부 또는 일부의 임대차에 관하여 적용한다\' 라고 규정하고 있습니다. 주거용 건물인지는 공부상의 표시에 불구하고 그 실지용도에 따라서 정하여야 합니다(대법원 1986. 1. 21. 선고 85다카1367 판결). 이 사건의 경우 숙박업소임에도 불구하고 해당 부분을 주거용으로 이용하고 있으므로 그 실지용도에 따라 주거용 건물로 판단하여 주임법이 적용대상에 포함시킬 수 있을것입니다.\r\n                   \t',
 '부부재산계약에 대하여는 「민법」 제829조가 규정하고 있습니다. 구체적으로 ① 부부가 혼인성립 전에 그 재산에 관하여 따로 약정을 하지 아니한 때에는 그 재산관계는 본관 중 다음 각 조에 정하는 바(부부재산별산제)에 의하며, ② 부부가 혼인성립 전에 그 재산에 관하여 약정한 때에는 혼인중 이를 변경하지 못한다. 그러나 정당한 사유가 있는 때에는 법원의 허가를 얻어 변경할 수 있으며, ③ 전항의 약정에 의하여 부부의 일방이 다른 일방의 재산을 관리하는 경우에 부적당한 관리로 인하여 그 재산을 위태하게 한 때에는 다른 일방은 자기가 관리할 것을 법원에 청구할 수 있고 그 재산이 부부의 공유인 때에는 그 분할을 청구할 수 있고, ④ 부부가 그 재산에 관하여 따로 약정을 한 때에는 혼인성립까지에 그 등기를 하지 아니하면 이로써 부부의 승계인 또는 제삼자에게 대항하지 못하며, ⑤ 제2항, 제3항의 규정이나 약정에 의하여 관리자를 변경하거나 공유재산을 분할하였을 때에는 그 등기를 하지 아니하면 이로써 부부의 승계인 또는 제삼자에게 대항하지 못한다고 규정되어 있습니다.\r따라서 혼인 전에 甲과 부부재산에 대한 계약을 미리 체결할 수 있으며, 원칙적으로 혼인 중에 계약 내용을 변경할 수는 없고, 계약 내용 관련하여 등기 등 공시절차가 이루어져야 

In [ ]:
return_answer('녹음 증언의 효력')

[0.8343057477722892, 0.8120130266414524, 0.7491638705182313, 0.7272394484607243, 0.7186117494389259]


['판례는 “수사기관이 아닌 사인(私人)이 피고인 아닌 사람과의 대화내용을 녹음한 녹음테이프는 「형사소송법」제311조, 제312조 규정 이외의 피고인 아닌 자의 진술을 기재한 서류와 다를 바 없으므로, 피고인이 그 녹음테이프를 증거로 할 수 있음에 동의하지 아니하는 이상 그 증거능력을 부여하기 위해서는 첫째, 녹음테이프가 원본이거나 원본으로부터 복사한 사본일 경우에는 복사과정에서 편집되는 등의 인위적 개작 없이 원본의 내용 그대로 복사된 사본일 것, 둘째 「형사소송법」제313조 제1항 에 따라 공판준비나 공판기일에서 원진술자의 진술에 의하여 그 녹음테이프에 녹음된 각자의 진술내용이 자신이 진술한 대로 녹음된 것이라는 점이 인정되어야 할 것이다(대법원 2011. 9. 8. 선고 2010도7497 판결 ; 대법원 2005.2.18.선고 2004도6323판결 등 참조).”라고 판시합니다.\r\n                   \t',
 '그 대화내용을 녹음한 원본이거나 혹은 원본으로부터 복사한 사본일 경우에는 복사과정에서 편집되는 등의 인위적 개작 없이 원본의 내용 그대로 복사된 사본임이 입증되어야만 하고, 그러한 입증이 없는 경우에는 쉽게 그 증거능력을 인정할 수 없다.',
 '판례는 “대화내용을 녹음한 파일 등의 전자매체는 성질상 작성자나 진술자의 서명 혹은 날인이 없을 뿐만 아니라, 녹음자의 의도나 특정한 기술에 의하여 내용이 편집ㆍ조작될 위험성이 있음을 고려하여 대화 내용을 녹음한 원본이거나 혹은 원본으로부터 복사한 사본일 경우에는 복사 과정에서 편집되는 등 인위적 개작 없이 원본의 내용 그대로 복사된 사본임이 입증되어야만 하고, 그러한 입증이 없는 경우에는 쉽게 그 증거능력을 인정할 수 없다. 그리고 증거로 제출된 녹음파일이 대화 내용을 녹음한 원본이거나 혹은 복사 과정에서 편집되는 등 인위적 개작 없이 원본 내용을 그대로 복사한 사본이라는 점은 녹음파일의 생성과 전달 및 보관 등의 절차에 관여한 사람의 증언이나 진술, 원본이나 사본 파일 생성 직후의 해

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/파일위치/pan_qna.csv')
n =10
sample = df.sample(n)
for i in sample.index:
  print(sample['question'][i])
  print(sample['answer'][i], '\n')

저명한 상품표지가 타인에 의하여 영업표지로 사용되는 경우에도 ‘식별력의 손상’이 생기는가?
‘식별력의 손상’은 ‘특정한 표지가 상품표지나 영업표지로서의 출처표시 기능이 손상되는 것’을 의미하는 것으로 해석함이 상당하며, 이러한 식별력의 손상은 저명한 상품표지가 다른 사람에 의하여 영업표지로 사용되는 경우에도 생긴다. 

게임물 관련 사업자가 제공 또는 승인하지 아니한 게임물을 제작, 배급, 제공 또는 알선해도 되는가?
인터넷 온라인 게임 ‘C’는 게임물 관련사업자인 (주)D가 제작, 배급, 제공하는 게임물 이고, 누구든지 게임물 관련사업자가 제공 또는 승인하지 아니한 게임물을 제작, 배급, 제공 또는 알선하는 행위를 하여서는 아니 된다. 

교차로에 횡형삼색등 신호기가 설치되어 있으나 달리 비보호좌회전 표지가 없는 경우, 녹색 등화시 유턴하여 진행하였다면 같은 진행방향의 후방차량에 대하여도 신호위반의 책임을 지나요?
같은 진행방향의 후방차량에 대하여도 책임을 지게 됩니다. 

이사회 내 위원회 위원는 음성 송수신 장치에 의해서도 결의참가가 가능하나요?
네. 이사회 내 위원회 위원는 음성 송수신 장치에 의해서도 결의참가가 가능합니다. 

공무수행사인의 경우 청탁금지법 제10조의 외부강의등 사례금 수수 제한 규정이 적용되나요?
청탁금지법의 적용대상이 아닙니다. 

부동산에 대한 강제집행정지 신청사건에서 담보제공명령을 받은 당사자가 아닌 제3자가 당사자를 대신하여 담보를 공탁한다는 취지를 공탁서에 기재함으로써 당사자를 위한 담보를 제공하였다면 제3자의 담보제공행위는 유효한가?
부동산에 대한 강제집행정지 신청사건에서 반드시 담보제공명령을 받은 당사자만이 담보를 제공하여야 한다고 볼 근거는 없으므로, 제3자 역시 당사자를 위한 담보를 제공할 수 있다. 

학교생활기록부 용어정의에서 사용자의 의미는 무엇인가요?
질의하신 사안의 경우, 학교에서의 ‘사용자'라 함은 실제로 학교생활기록부의 전산 업무를 총괄하도록 학교장의 임명을 받은 교사와 학생의 학교생활기록부를 직접 전산으

In [ ]:
return_answer('공무수행사도 청탁금지법의 대상이 되는지?')

[0.8380726212357431, 0.8336703179645535, 0.833554625921313, 0.8241813244489404, 0.8223933476459205]


['네 청탁금지법의 적용대상입니다.',
 '청탁금지법에서 열거한 14가지 청탁만 규율대상에 해당합니다.',
 '행정기관 계약직 직원의 경우 공무원 신분이 아니라면 공직자등에 해당하지 않는 반면, 공직유관단체 계약직 직원은 당해 기관과 직접 근로계약을 체결했다면 청탁금지법상 공직자등에 해당합니다.',
 '「민사소송 등 인지법 」 제3조는 “항소장에는 제2조에 따른 금액의 1.5배액의 인지를, 상고장(대법원에 제출하는 소장을 포함)에는 제2조에 따른 금액의 2배액의 인지를 붙여야 한다.”라고 규정하고 있으므로, 항소장 제출시에는 제1심 소장의 첩부인지액의 1.5배액의 인지를 첩부하여야 합니다.그런데  「민사소송등인지법 」 제3조가 상소인을 제1심 원고에 비하여 불합리하게 차별하는 것이 아닌지, 또한 위 규정이 무자력자의 재판청구권을 불합리하게 제한 또는 차별하는 것이 아닌지에 관하여 판례는 “인지대가 단순히 국가의 역무에 비례한 비용으로서의 성질만을 가지는 것만이 아니라 남소(濫訴) 또는 남상소(濫上訴)에 따른 법원의 국민의 권리보호기능을 저하하게 하는 것을 막게 하는 기능을 아울러 가지고 있다는 그 본래의 목적과 항소장 또는 상고장의 인지액의 기준이 되는 제1심 인지액 자체가 소가(訴價)에서 차지하는 비율이 비교적 근소하게 규정되어 있음을 함께 고려하면, 제1심 원고에 비하여 항소인이나 상고인은 이미 소송절차상 보장된 공격방어방법을 이용하여 법원의 재판을 한 두 번 거친 자라는 실질적인 차이에 근거하여 제1심 소장, 항소장 및 상고장에 붙여야 할 인지액에 차등을 두어 단계적으로 인지액을 인상하도록 규정한 민사소송등인지법 제3조는 합리적 근거가 없거나 그 정도가 지나쳐서 상소인을 부당하게 차별하는 것이라고 할 수 없다.”라고 하면서 또한, “민사소송법은 자력이 부족한 자를 위하여 소송구조제도를 마련하여 그 당사자가 패소할 것이 명백하여 항소 또는 상고의 실질적 이익이 없다고 인정되지 않는 한 항소장 또는 상고장에 인지를 붙이지 아니하고도 항소심 또는 상고심의 재판

In [ ]:
return_answer('식별력의 손상의 의미')

[0.7336437807186525, 0.7270901344350402, 0.7127915678536391, 0.7109068405730339, 0.7109068405730339]


['등록부의 기록이 법률상 무효인 것이거나 그 기록에 착오 또는 누락이 있는 경우이므로 가족관계등록법 제18조에 다른 직권정정의 사유이며, 한편 동법 제104조에 따라 관할 가정법원에 가족관계등록부 정정허가신청을 하여 법원의 허가를 받아 시(구)·읍·면의 장에게 정정신청을 하는 방법으로도 가능합니다.\r\n                   \t',
 "소유권에 기한 방해배제청구권에 있어서 '방해'라 함은 현재에도 지속되고 있는 침해를 의미하고, 법익 침해가 과거에 일어나서 이미 종결된 경우에 해당하는 '손해'의 개념과는 다르다 할 것이어서, 소유권에 기한 방해배제청구권은 방해결과의 제거를 내용으로 하는 것이 되어서는 아니 되며(이는 손해배상의 영역에 해당한다 할 것이다) 현재 계속되고 있는 방해의 원인을 제거하는 것을 내용으로 한다. 쓰레기 매립으로 조성한 토지에 소유권자가 매립에 동의하지 않은 쓰레기가 매립되어 있다 하더라도 이는 과거의 위법한 매립공사로 인하여 생긴 결과로서 소유권자가 입은 손해에 해당한다 할 것일 뿐, 그 쓰레기가 현재 소유권에 대하여 별도의 침해를 지속하고 있다고 볼 수 없다는 이유로 소유권에 기한 방해배제청구권을 행사할 수 없다.(대법원 2003. 3. 28. 선고 2003다5917 판결)따라서 소유물방해배제청구권에 기한 폐기물 수거는 구할 수 없고 손해배상으로 해결하여야 합니다.\r\n                   \t",
 '형법 제152조는 “법률에 의하여 선서한 증인이 허위의 진술을 한 사람을 위증죄로 처벌하고 있습니다. 위 사례와 같이 甲이 자신의 기억에 반하는 증언을 하였지만, 그 증언이 진실일 경우 甲이 한 증언이 위증죄에서 말하는 허위진술에 해당하는지 여부가 문제됩니다.  이와 관련하여 대법원은 “위증죄에 있어서의 허위의 공술이란 증인이 자기의 기억에 반하는 사실을 진술하는 것을 말하는 것으로서 그 내용이 객관적 사실과 부합한다고 하여도 위증죄의 성립에 장애가 되지 않는다”라고 판단하였습니다(대법원 1989.01.17

In [ ]:
return_answer('비보호 좌회전 표시가 없는 교차로에서 초록불이 켜졌을 때 유턴하였다면 뒤쪽 차량에 대해 책임을 지는지 ')

[0.8257910682230754, 0.8257095563618172, 0.7990221396616588, 0.7936984333019458, 0.7891313997174699]


['같은 진행방향의 후방차량에 대하여도 책임을 지게 됩니다.',
 '신호위반의 책임은 인정되지 않습니다.',
 '교차로에 녹색, 황색 및 적색의 삼색등화만이 나오는 신호기가 설치되어 있고 달리 비보호좌회전 표시나 유턴을 허용하는 표시가 없는 경우에 차마의 좌회전 또는 유턴은 원칙적으로 허용되지 않는다고 보아야 한다.',
 '원칙적으로 허용되지 않습니다.',
 '횡형삼색등신호기가 교차로의 대각선 지점에 있지 아니하고 교차로에 연이어 있는 횡단보도상에 보행자 신호기와 함께 설치되어 있을 경우 이는 횡단보도상을 통행하는 보행자를 보호하기 의하여 차량들에 대한 횡단보도에 진입 또는 정지를 지시하는 신호기로 보아야 한다.']